In [1]:
from google.colab import files

uploaded = files.upload()

Saving dc_md_va_flash_floods_1996_present.csv to dc_md_va_flash_floods_1996_present.csv


In [27]:
import pandas as pd
import numpy as np

df = pd.read_csv('dc_md_va_flash_floods_1996_present.csv')

df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,199601,19,1100,199601,19,1300,2403644,5541358,DISTRICT OF COLUMBIA,11,...,NaN,NaN,ALL,NaN,NaN,NaN,NaN,"An unusually intense squall line, feeding on a...",NaN,PDC
1,199606,18,30,199606,18,200,1014286,5561204,DISTRICT OF COLUMBIA,11,...,NaN,NaN,N PORTION,NaN,NaN,NaN,NaN,Intense thunderstorms moved through the northe...,NaN,PDC
2,199606,20,2200,199606,20,2300,2040906,5561207,DISTRICT OF COLUMBIA,11,...,NaN,NaN,NORTHWEST,NaN,NaN,NaN,NaN,A rapidly developing thunderstorm knocked down...,NaN,PDC
3,199609,6,1200,199609,6,1800,2049837,5572405,DISTRICT OF COLUMBIA,11,...,NaN,NaN,DOWNTOWN,NaN,NaN,NaN,NaN,Feeder bands of torrential tropical rains asso...,NaN,PDC
4,199611,8,1600,199611,8,1715,2049872,5572790,DISTRICT OF COLUMBIA,11,...,NaN,NaN,CITYWIDE,NaN,NaN,NaN,NaN,Several roads were briefly closed after torren...,NaN,PDC


In [54]:
df.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE', 'date',
       'n_episodes', 'n_events', 'damage_property', 'injuries', 'fatalities',
       'label', 'month', 'season', 'date_dt', 'prev_damage_date',
   

In [33]:
df['CZ_NAME'].unique()

array(['DISTRICT OF COLUMBIA', 'WYTHE', 'CAMPBELL', 'TAZEWELL', 'CARROLL',
       'CRAIG', 'PULASKI', 'ROANOKE', 'MONTGOMERY', 'ARLINGTON',
       'PITTSYLVANIA', 'HIGHLAND', 'ROCKBRIDGE', 'SMYTH', 'RAPPAHANNOCK',
       'BLAND', 'GILES', 'SPOTSYLVANIA', 'WISE', 'FREDERICK',
       'SHENANDOAH', 'PATRICK', 'NELSON', 'ALBEMARLE', 'GREENE',
       'MADISON', 'PAGE', 'STAFFORD', 'ALLEGHANY', 'WARREN', 'SCOTT',
       'ROCKINGHAM', 'APPOMATTOX', 'LOUDOUN', 'RUSSELL', 'DICKENSON',
       'AUGUSTA', 'ORANGE', 'BUCHANAN', 'HENRY', 'FRANKLIN',
       'SOUTHAMPTON', 'LEE', 'BEDFORD', 'FAUQUIER', 'CULPEPER',
       'KING WILLIAM', 'CLARKE', 'FAIRFAX', 'AMHERST', 'BOTETOURT',
       'BATH', 'GRAYSON', 'LOUISA', 'DINWIDDIE', 'HALIFAX',
       'KING AND QUEEN', 'PRINCE WILLIAM', 'CHESAPEAKE (C)',
       'VIRGINIA BEACH (C)', 'NORFOLK (C)', 'CHARLOTTE', 'FLOYD',
       'BUCKINGHAM', 'COVINGTON (C)', 'WASHINGTON', 'SALEM',
       'ROANOKE (C)', 'DANVILLE (C)', 'CHARLOTTESVILLE (C)',
       'KING GEOR

In [36]:
df['date'] = df['BEGIN_YEARMONTH'] * 100 + df['BEGIN_DAY']
#df['CZ_NAME'] = df['CZ_NAME'].str.replace(r'\s*\(C\)', '', regex=True).str.strip()


In [37]:
#Investigating the Fairfax and Fairfax (C) issue
#unique_combos = df[['date', 'CZ_NAME']].drop_duplicates()
#print(unique_combos.loc[unique_combos['CZ_NAME'] == 'FAIRFAX'])
#print(unique_combos.loc[unique_combos['CZ_NAME'] == 'FAIRFAX (C)'])

#print(df['CZ_NAME'].unique())

In [38]:
df['n_episodes'] = df.groupby(['CZ_NAME', 'date'])['EPISODE_ID'].transform('nunique')
df['n_events'] = df.groupby(['CZ_NAME', 'date'])['EVENT_ID'].transform('nunique')

In [39]:
#summary_df = df.groupby(['CZ_NAME', 'date']).agg(
#    n_events=('EVENT_ID', 'nunique')
#).reset_index()
#
#top_10 = summary_df.sort_values(by='n_events', ascending=False).head(10)

#print(top_10)

In [40]:
def parse_damage(val):
    if pd.isna(val) or val == '':
        return np.nan
    val = str(val).strip().upper().replace('$', '')
    if val.endswith('K'):
        return float(val[:-1]) * 1000
    elif val.endswith('M'):
        return float(val[:-1]) * 1000000
    try:
        return float(val)
    except:
        return np.nan

df['damage_property'] = df['DAMAGE_PROPERTY'].apply(parse_damage)

In [41]:
df['injuries'] = df[['INJURIES_DIRECT', 'INJURIES_INDIRECT']].sum(axis=1, skipna=True)
df['fatalities'] = df[['DEATHS_DIRECT', 'DEATHS_INDIRECT']].sum(axis=1, skipna=True)

In [42]:
df['label'] = None

df.loc[
    (df['injuries'] > 0) |
    (df['fatalities'] > 0) |
    (df['damage_property'] > 0),
    'label'
] = 1

df.loc[
    (df['injuries'] == 0) &
    (df['fatalities'] == 0) &
    (df['damage_property'] == 0),
    'label'
] = 0

In [43]:
exclude_mask = (
    df[['injuries', 'fatalities', 'damage_property']].isna().any(axis=1) &
    (df['injuries'].fillna(0) == 0) &
    (df['fatalities'].fillna(0) == 0) &
    (df['damage_property'].fillna(0) == 0)
)

df = df[~exclude_mask]

In [44]:
df.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE', 'date',
       'n_episodes', 'n_events', 'damage_property', 'injuries', 'fatalities',
       'label'],
      dtype='object')

In [45]:
#new = df[['END_YEARMONTH',
#       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
#      'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
#       'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'DAMAGE_PROPERTY','FLOOD_CAUSE', 'CATEGORY', 'BEGIN_RANGE',
#       'date','n_episodes', 'n_events', 'damage_property', 'injuries', 'fatalities',
#       'label']]
#new.shape

In [46]:
df = df.copy()
df['month'] = (df['date'] % 10000) // 100

In [47]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

df['season'] = df['month'].apply(get_season)

In [48]:
def int_to_datetime(x):
    s = str(int(x))
    return pd.Timestamp(year=int(s[:4]), month=int(s[4:6]), day=int(s[6:8]))

df['date_dt'] = df['date'].apply(int_to_datetime)

df = df.sort_values(['CZ_NAME', 'date_dt'])

df_damage = df[df['label'] == 1].copy()

df_damage['prev_damage_date'] = df_damage.groupby('CZ_NAME')['date_dt'].shift()

df = df.merge(
    df_damage[['CZ_NAME', 'date_dt', 'prev_damage_date']],
    how='left',
    on=['CZ_NAME', 'date_dt']
)

df['prev_damage_date_filled'] = df.groupby('CZ_NAME')['prev_damage_date'].ffill()

df['days_since_last_flood'] = (df['date_dt'] - df['prev_damage_date_filled']).dt.days

df['days_since_last_flood'] = df['days_since_last_flood'].fillna(-1).astype(int)

In [49]:
grouped = df.groupby(['CZ_NAME', 'date'])

county = 'FAIRFAX'
date = 20190708

group = grouped.get_group((county, date))
print(group)

      BEGIN_YEARMONTH  BEGIN_DAY  BEGIN_TIME  END_YEARMONTH  END_DAY  \
1850           201907          8         901         201907        8   
1851           201907          8         915         201907        8   
1852           201907          8         915         201907        8   
1853           201907          8         915         201907        8   
1854           201907          8         915         201907        8   
1855           201907          8         915         201907        8   
1856           201907          8         915         201907        8   
1857           201907          8         915         201907        8   
1858           201907          8         915         201907        8   
1859           201907          8         915         201907        8   
1860           201907          8         915         201907        8   
1861           201907          8         915         201907        8   
1862           201907          8         915         201907     

In [60]:
num_groups = df.groupby(['CZ_NAME', 'date']).ngroups
print(num_groups)

episode_counts = df.groupby(['CZ_NAME', 'date'])['EPISODE_ID'].nunique().reset_index()

multiple_episodes = episode_counts[episode_counts['EPISODE_ID'] > 1]

print(multiple_episodes)

2585
                 CZ_NAME      date  EPISODE_ID
114              AMHERST  20180527           2
219            BALTIMORE  19960619           2
258            BALTIMORE  20130610           2
314   BALTIMORE CITY (C)  19960619           2
361              BEDFORD  20030809           2
468           BUCKINGHAM  20060627           2
495             CAMPBELL  19960119           2
500             CAMPBELL  20070727           2
529             CAROLINE  19990916           2
534             CAROLINE  20090822           2
550              CARROLL  19960119           2
551              CARROLL  19960619           2
555              CARROLL  19980108           2
976                FLOYD  20181011           2
1018           FREDERICK  19960119           2
1021           FREDERICK  19960619           2
1024           FREDERICK  19960906           2
1028           FREDERICK  19980128           2
1030           FREDERICK  19980320           2
1051           FREDERICK  20120918           2
1346    

In [51]:
grouped = df.groupby(['CZ_NAME', 'date'])

county = 'BALTIMORE'
date = 19960619

group = grouped.get_group((county, date))
print(group)

     BEGIN_YEARMONTH  BEGIN_DAY  BEGIN_TIME  END_YEARMONTH  END_DAY  END_TIME  \
391           199606         19        1115         199606       19      1500   
392           199606         19        1115         199606       19      1500   
393           199606         19        2245         199606       20       100   
394           199606         19        2245         199606       20       100   

     EPISODE_ID  EVENT_ID     STATE  STATE_FIPS  ...  damage_property  \
391     1014249   5560610  MARYLAND          24  ...          25000.0   
392     1014249   5560610  MARYLAND          24  ...          25000.0   
393     2040875   5561170  MARYLAND          24  ...          25000.0   
394     2040875   5561170  MARYLAND          24  ...          25000.0   

    injuries fatalities label  month  season    date_dt prev_damage_date  \
391        0          2     1      6  Summer 1996-06-19       1996-06-17   
392        0          2     1      6  Summer 1996-06-19       1996-06-19   


In [52]:
df['n_episodes']
episode_counts = df.groupby(['CZ_NAME', 'date'])['EPISODE_ID'].nunique()

count_distribution = episode_counts.value_counts().sort_index()

print(count_distribution)

EPISODE_ID
1    2554
2      30
3       1
Name: count, dtype: int64


In [53]:
df['CZ_NAME'].unique()

array(['ACCOMACK', 'ALBEMARLE', 'ALEXANDRIA (C)', 'ALLEGANY', 'ALLEGHANY',
       'AMELIA', 'AMHERST', 'ANNE ARUNDEL', 'APPOMATTOX', 'ARLINGTON',
       'AUGUSTA', 'BALTIMORE', 'BALTIMORE CITY (C)', 'BATH', 'BEDFORD',
       'BLAND', 'BOTETOURT', 'BRISTOL (C)', 'BRUNSWICK', 'BUCHANAN',
       'BUCKINGHAM', 'BUENA VISTA (C)', 'CALVERT', 'CAMPBELL', 'CAROLINE',
       'CARROLL', 'CECIL', 'CHARLES', 'CHARLES CITY (C)', 'CHARLOTTE',
       'CHARLOTTESVILLE (C)', 'CHESAPEAKE (C)', 'CHESTERFIELD', 'CLARKE',
       'COLONIAL HEIGHTS (C)', 'COVINGTON (C)', 'CRAIG', 'CULPEPER',
       'CUMBERLAND', 'DANVILLE (C)', 'DICKENSON', 'DINWIDDIE',
       'DISTRICT OF COLUMBIA', 'DORCHESTER', 'EMPORIA', 'ESSEX',
       'FAIRFAX', 'FAIRFAX (C)', 'FALLS CHURCH (C)', 'FAUQUIER', 'FLOYD',
       'FLUVANNA', 'FRANKLIN', 'FRANKLIN (C)', 'FREDERICK',
       'FREDERICKSBURG (C)', 'GALAX (C)', 'GARRETT', 'GILES',
       'GLOUCESTER', 'GOOCHLAND', 'GRAYSON', 'GREENE', 'GREENSVILLE',
       'HALIFAX', 'HAMPTON (C)

In [62]:
columns_to_keep = [
    'CZ_NAME', 'n_episodes', 'n_events', 'damage_property',
    'injuries', 'fatalities', 'label', 'month', 'season',
    'date_dt', 'prev_damage_date'
]

df_copy = df[columns_to_keep].copy()

df_copy.head()


,CZ_NAME,n_episodes,n_events,damage_property,injuries,fatalities,label,month,season,date_dt,prev_damage_date
0,ACCOMACK,1,1,0.0,0,0,0,8,Summer,2000-08-04,NaT
1,ACCOMACK,1,3,0.0,0,0,0,7,Summer,2009-07-27,NaT
2,ACCOMACK,1,3,0.0,0,0,0,7,Summer,2009-07-27,NaT
3,ACCOMACK,1,3,0.0,0,0,0,7,Summer,2009-07-27,NaT
4,ACCOMACK,1,1,0.0,0,0,0,6,Summer,2011-06-19,NaT


In [65]:
grouped_df = df.groupby(['CZ_NAME', 'date_dt'], as_index=False).agg({
    'injuries': 'sum',
    'fatalities': 'sum',
    'damage_property': 'sum',
    'n_episodes': 'first',
    'n_events': 'first',
    'label': 'first',
    'month': 'first',
    'season': 'first',
    'prev_damage_date': 'first'
})

grouped_df

,CZ_NAME,date_dt,injuries,fatalities,damage_property,n_episodes,n_events,label,month,season,prev_damage_date
0,ACCOMACK,2000-08-04,0,0,0.0,1,1,0,8,Summer,NaT
1,ACCOMACK,2009-07-27,0,0,0.0,1,3,0,7,Summer,NaT
2,ACCOMACK,2011-06-19,0,0,0.0,1,1,0,6,Summer,NaT
3,ACCOMACK,2012-07-14,0,0,0.0,1,1,0,7,Summer,NaT
4,ACCOMACK,2012-08-25,0,0,0.0,1,2,0,8,Summer,NaT
...,...,...,...,...,...,...,...,...,...,...,...
2580,YORK,2020-12-24,0,0,0.0,1,1,0,12,Winter,NaT
2581,YORK,2022-07-15,0,0,0.0,1,1,0,7,Summer,NaT
2582,YORK,2023-09-10,0,0,0.0,1,1,0,9,Fall,NaT
2583,YORK,2024-07-12,0,0,0.0,1,2,0,7,Summer,NaT
